In [1]:
import ollama
import chromadb
import pandas as pd
import chromadb.utils.embedding_functions as embedding_functions
import config

/tmp/ipykernel_3202211/3779128832.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('data/recycling guide.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
    
data = {'text': lines}
df = pd.DataFrame(data)

df.head()

,text
0,재활용품은 재질별로 분류하여 분리수거함 및 별도의 전용수거함으로 배출한다.
1,대형 폐가전제품은 무상 방문 수거를 신청한다. 소형 폐가전제품도 같이 수거 가능하다.
2,"대형폐기물은 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재..."
3,음식물류폐기물는 RFID 기반 음식물 폐기장을 사용하거나 종량제봉투에 담아 배출한다...
4,"유해폐기물로 구분된 폐전건지, 형광등, 폐의약품은 전용수거함을 이용한다."


In [3]:
df.tail()

,text
243,식품보조제 콜라겐가루가 남았는데 어떻게 버려야 하나요? 일반쓰레기로 버려야 하나요 ...
244,섬유유연제가 걸쭉하니 굳어가네요. 남은 양이 꽤 많은데 하수구에 버리자니 막힐 것 ...
245,다 쓴 프린터잉크 (페카트리지)는 어떻게 배출하나요?? 저희 읍에서는 종량제봉투에 ...
246,최근 아이스팩이 심각한 수질오염을 일으킬 수 있다는 얘기를 접했습니다. 단순히 종량...
247,"택배 올 때 비닐로 된 포장재가 있습니다. 뽁뽁이 말고요~ 연두빛 비닐인데, 산화 ..."


In [4]:
num_rows = df.shape[0]
print("DataFrame의 줄 개수:", num_rows)

DataFrame의 줄 개수: 248


In [5]:
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=config.API['hugging_face'],
    model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"
    #model_name="jhgan/ko-sbert-sts"
)

client = chromadb.PersistentClient(path="DB_recycling")
collection = client.get_or_create_collection(name="recycling_chroma", embedding_function=huggingface_ef)

docs = df["text"].tolist()
ids = [str(x) for x in df.index.tolist()]

collection.add(
    documents = docs,
    ids = ids
)

In [6]:

results = collection.query(
    query_texts=["자전거는 어떻게 버려?"],
    n_results=5
)

res = "\n".join(str(item) for item in results['documents'][0])

print(res)

대형폐기물인 자전거는 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재활용센터에 방문해 처리할 수 있다.
개나 고양이와 같이 반려동물 사료는 어떻게 버리나요? 음식물류 폐기물은 음식재료 또는 음식물의 생산･유통･가공･조리･보관･소비과정 등에서 발생되는 쓰레기와 남겨서 버려지는 음식물 등을 말합니다. 반려동물의 사료는 음식물류 폐기물로 보기는 어려울 것 같습니다. 일반쓰레기로 배출하여 주시기 바랍니다.
의류인 기타의류는 지자체 또는 민간 재활용사업자가 비치한 폐의류 전용수거함에 배출한다. 전용수거함이 없는 지역 등에서는 물기에 젖지 않도록 마대 등에 담거나 묶어서 배출한다.
튀김을 하고 남은 폐식용유는 어떻게 버려야하나요? 저희 아파트는 폐식용유 전용수거함이 없습니다... 폐식용유 수거함은 지역에 따라 관리되고 있기 때문에 구청 또는 주민센터에 문의하신 후 배출하시기 바랍니다.
회사에서 사용중인 스마트포인터를 버려야하는데 어떻게 버려야하나요? 전기전자제품은 현재 27종이 환경성보장제도 의무재활용 대상에 해당되는데요. 문의하신 스마트포인터는 여기에 해당되지는 않습니다. 그러나 지자체 마다 소형전기전자제품을 재활용하는 경우도 있으니 구청이나 주민센터 등에 문의하신 후 배출하여 주시기 바랍니다.


In [7]:
sys_prompt = f'''
너는 재활용 전문가로 활동하고 있어.
사람들이 재활용 방법을 물어보면 너는 주어진 Context를 바탕으로 질문에 해당하는 내용만 짧게 요약해서 답해줘야해.

Context: {res}
'''

prompt = '자전거는 어떻게 버려?'

messages = sys_prompt + '\n' + prompt

output = ollama.generate(
  model="Llama3_ko",
  prompt=messages
)

print(output['response'])

 자전거를 사용하시다가 고장이 나면 정비업체나 대리점에서 수리를 하셔야 합니다. 사용하지 않는 중고 자전거의 경우에는 지자체 또는 민간재활용사업자가 비치한 폐자전거 전용수거함에 배출하시거나, 
